In [13]:
###Rather than reinitializing maybe update keys
class keys:
    def __init__(self, event= None):
        self.last_pressed = ""
        self.string = "Pressed:"
        if event.key == pygame.K_UP:
            self.UP = True
            self.string += "UP"
        else:
            self.UP = False
        if event.key == pygame.K_DOWN:
            self.DOWN = True
            self.string += "DOWN"
        else:
            self.DOWN = False
        if event.key == pygame.K_RIGHT:
            self.RIGHT = True
            self.string += "RIGHT"
            self.last_pressed = "RIGHT"
        else:
            self.RIGHT = False    
        if event.key == pygame.K_LEFT:
            self.LEFT = True
            self.string += "LEFT"
            self.last_pressed = "LEFT"
        else:
            self.LEFT = False 
 
    def __str__(self):
        return self.string
        
        
def xy_position_to_pixels(Rectangle, window_size, virtual_window):
    """Takes:In-game coordinate rectangle
    Converts in game coordinates to pixels. Also, note that the in-game position refers to the LOWER RIGHT hand
    corner of the sprite. 
    Returns: rectangle in pixel coordinates"""
    x = Rectangle[0]
    y = Rectangle[1]
    width = Rectangle[2]
    height = Rectangle[3]
    pixel_x = int(float(x/virtual_window[0])*window_size[0])
    pixel_y = window_size[1] - int(float(y/virtual_window[1])*window_size[1])
    pixel_width = int(float(width/virtual_window[0])*window_size[0])
    pixel_height = int(float(height/virtual_window[1])*window_size[1])
    pixel_y -= pixel_height
    return (pixel_x, pixel_y, pixel_width, pixel_height)



class position:
    def __init__(self, current_x = 0, current_y = 0, current_xs = 0, current_ys = 0, floor = 0, button=None):
        self.current_x = current_x
        self.current_y = current_y
        self.current_xs = current_xs
        self.current_ys = current_ys
        self.button = button
        self.floor = floor
        self.last_pressed = None
        self.jump = False
    def __str__(self):
        return str(self.current_x)+" "+str(self.current_y)+" "+str(self.current_xs)+" "+str(self.current_ys)
    
    def give_next_position(self): #fix button 
        #print("The give next position function has run.")
        #print("Button:", button)
        if self.current_y == self.floor:
            self.current_ys = 0
            self.jump = False
        if not self.jump:    
            self.current_xs = 0
        if self.current_y > self.floor:
            self.current_ys -= .5
            
        if self.button != None: #FOUND PROBLEM ONLY UPDATES WITH BUTTON PRESS
            #print("AH I SENSE MOVEMENT!")
            if self.current_y == self.floor:
                if self.button.UP:
                    self.current_ys = 10 
                    if self.last_pressed == "RIGHT":
                        self.current_xs = 1
                        self.jump = True
                    elif self.last_pressed == "LEFT":
                        self.current_xs = -1
                        self.jump = True
            if self.button.RIGHT:
                self.current_xs = 1
            elif self.button.LEFT:
                self.current_xs = -1
            self.last_pressed = self.button.last_pressed
            
        self.current_x += self.current_xs
        self.current_y += self.current_ys
        
        if self.current_y < self.floor:
            self.current_y = self.floor
        

In [44]:
###PLATFORMS###
class platform:
    def __init__(self, x1, x2, height):
        self.rightX = x1
        self.leftX = x2
        self.height = height

platform_generation_list = [(120,200,50)]

def generate_list_of_platform_objects(platform_generation_list):
    list_of_platforms = []
    for i in platform_generation_list:
        platform_temp = platform(i[0], i[1], i[2])
        list_of_platforms.append(platform_temp)
    return list_of_platforms

def generate_list_of_platforms_on_screen(list_of_platforms, virtual_window):
    list_of_platforms.sort(key = lambda plat: plat.leftX) ###SORT BY LEFTX
    list_of_platforms_on_screen = []
    for i in list_of_platforms:
        if i.rightX < 0:
            continue
        list_of_platforms_on_screen.append(i)
        if i.leftX > virtual_window[0]:
            break
    return list_of_platforms_on_screen

def blit_platform(platform):
    pass



###BLOCKS###
class block:
    def __init__(self, x, y, width, height):
        self.rightX = x
        self.leftX = x+width
        self.top = y + height
        self.bot = y
        

def generate_list_of_block_objects(block_generation_list):
    list_of_blocks = []
    for i in block_generation_list:
        block_temp = block(i[0],i[1], i[2], i[3])
        list_of_blocks.append(block_temp)
    return list_of_blocks

def generate_list_of_blocks_on_screen(list_of_blocks, virtual_window):
    list_of_blocks.sort(key = lambda bl: bl.leftX) ###SORT BY LEFTX
    list_of_blocks_on_screen = []
    for i in list_of_blocks:
        if i.rightX < 0:
            continue
        list_of_blocks_on_screen.append(i)
        if i.leftX > virtual_window[0]:
            break
    return list_of_blocks_on_screen

def blit_block(block, window_size, virtual_window):
    rectangle = (block.leftX, block.bot, block.rightX-block.leftX, block.top-block.bot)
    pixel_rectangle = xy_position_to_pixels(rectangle, window_size, virtual_window)
    pygame.draw.rect(screen, BLUE, pixel_rectangle)

In [47]:
import pygame

virtual_window = (300,200)

pygame.init()
window_size = (pygame.display.Info().current_w -50, pygame.display.Info().current_h - 50)
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Egg World")
done = False
clock = pygame.time.Clock()
BLUE = (0,0,255)

block_generation_list = [(0,0,300,20)]
list_of_blocks = generate_list_of_block_objects(block_generation_list)
platform_generation_list = [(120,200,50)]
list_of_platforms = generate_list_of_platform_objects(platform_generation_list)

current_position = position(current_y=  virtual_window[1])  



#timer_event = pygame.USEREVENT + 1
#pygame.time.set_timer(timer_event, 250) #makes it so that your not waiting for events to run
while not done:
    clock.tick(50)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                done = True
            #print("KEY DOWN!")
            current_position.button = keys(event)
            #print(current_position.button)
            break
        if event.type == pygame.KEYUP:
            #print("KEY UP!")
            current_position.button = None
            break
            
    blocks_on_screen = generate_list_of_blocks_on_screen(list_of_blocks, virtual_window)
    platforms_on_screen = generate_list_of_platforms_on_screen(list_of_platforms, virtual_window)
            
    #print("Outside loop sees ", current_position.button)
    current_position.give_next_position()
    screen.fill((0,0,50))
    ##floor
    for i in blocks_on_screen:
        blit_block(i, window_size, virtual_window)
    for i in platforms_on_screen:
        pass
        
    ##Character
    Rectangle = xy_position_to_pixels(
        (current_position.current_x, current_position.current_y,20,30), window_size, virtual_window)
    pygame.draw.rect(screen, BLUE, Rectangle)
    
    
    pygame.display.flip()
        
        
pygame.quit()

